# Weekly project 6
Today we will continue work from monday.
We will follow the style of last week.

Weekly project:
- You will need to implement your own k-means algorithm. (So you are not allowed to use the one in *sklearn*
- it should be able to cluster each of the different figures.
- Extend your k-means so it finds the optimal amount of clusters.
Challenge
- Implement the mean shift clustering algorithm


In [23]:
import numpy as np
import open3d as o3d
import copy
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
%matplotlib notebook


def draw_labels_on_model(pcl, labels):
    cmap = plt.get_cmap("tab20")
    pcl_temp = copy.deepcopy(pcl)
    max_label = labels.max()
    colors = cmap(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcl_temp.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcl_temp])

d = 4
mesh = o3d.geometry.TriangleMesh.create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_torus().translate((-d, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_moebius(twists=1).translate((0, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_moebius(twists=2).translate((d, -d, 0))

## apply k means on this
point_cloud = mesh.sample_points_uniformly(int(1e4))

In [2]:
#print(type(point_cloud))
point_cloud_array = np.asarray(point_cloud.points)
print(type(point_cloud_array))
print(point_cloud_array)



<class 'numpy.ndarray'>
[[-3.28501824e+00  8.60358667e-03 -3.33333333e-01]
 [-3.28482926e+00  5.65278680e-02 -3.33333333e-01]
 [-4.06344277e+00 -3.99459343e-01 -3.33333333e-01]
 ...
 [ 5.32553196e+00 -4.01506005e+00 -3.34967510e-03]
 [ 5.41279425e+00 -4.06641196e+00 -1.99476786e-02]
 [ 5.43896447e+00 -4.01864985e+00 -5.60172166e-03]]


In [ ]:
# This is the initial version of finding dist DONT RUN

#         for i in pcl_array:
#             dist = []
#             for j in centroids:
#                 dist.append(np.linalg.norm(i-j))
#             closest_center = np.argmin(dist)
#             labels.append(closest_center)
        #print(dicktionary)

In [38]:
def kmeans2(pcl_array, num_clusters, max_iter):
#     big = np.max(pcl_array)
#     small = np.min(pcl_array)
    centroids = np.random.uniform(0, 1, (num_clusters, 3))
    
    for k in range(max_iter):
        dist = cdist(point_cloud_array, centroids)
        labels = np.argmax(dist, 1)
        for l in range(len(centroids)):
            np_item = point_cloud_array[np.where(labels == l)]
            if np_item.size == 0:
                continue
            mean = np.mean(np_item, axis = 0)
            centroids[l] = mean
    return labels

labels = kmeans2(point_cloud_array, 6, 10)
draw_labels_on_model(point_cloud, labels)

#o3d.visualization.draw_geometries([point_cloud])

5
[[ 1.36811105e-01  4.26999711e-01 -2.30020689e-01]
 [-1.74522207e+00 -3.67375140e+00 -7.20380929e-03]
 [ 9.83447732e-03 -4.08533316e-02  7.56912431e-01]
 [-3.27501978e+00  4.72836724e-02  4.69977519e-02]
 [ 4.01839307e+00 -1.59056601e-01  1.03004070e-03]
 [ 6.31098376e-01  3.56102959e-01  6.62592684e-01]]
0
2
5
[[ 1.36811105e-01  4.26999711e-01 -2.30020689e-01]
 [ 2.60579593e+00  8.59985142e-01  1.67487828e-02]
 [ 9.83447732e-03 -4.08533316e-02  7.56912431e-01]
 [ 3.86509289e+00 -1.32771623e+00 -7.26288911e-03]
 [-3.18560381e+00 -2.99535898e+00  3.17514424e-04]
 [ 6.31098376e-01  3.56102959e-01  6.62592684e-01]]
0
2
5
[[ 1.36811105e-01  4.26999711e-01 -2.30020689e-01]
 [-1.94739611e+00 -4.41554536e+00 -1.89167751e-03]
 [ 9.83447732e-03 -4.08533316e-02  7.56912431e-01]
 [-3.68509128e+00 -2.21599026e+00  1.39421100e-03]
 [ 3.63797946e+00 -5.77957829e-01 -9.57816403e-04]
 [ 6.31098376e-01  3.56102959e-01  6.62592684e-01]]
2
5
[[ 6.99027861e-01 -4.92605491e+00 -9.72975636e-02]
 [ 1.31806

In [43]:
def my_kmeans(x, k):

    diff = 1
    cluster = np.zeros(x.shape[0]) #1000 in our case

    # select k random centroids
    random_indices = np.random.choice(len(x), size=k, replace=False) # we choose randomly k points in a len(x) extension
    centroids = x[random_indices, :]

    while diff:

      # for each observation
      for i, row in enumerate(x):

        mn_dist = float('inf')
        # dist of the point from all centroids
        for idx, centroid in enumerate(centroids):
          d = np.sqrt((centroid[0]-row[0])**2 + (centroid[1]-row[1])**2+ (centroid[2]-row[2])**2)

          # store closest centroid 
          if mn_dist > d:
            mn_dist = d
            cluster[i] = idx # classify each point on a cluster

      new_centroids = np.zeros((k,3))
      for j in range(k):
        cluster_aux = []
        for i in range(x.shape[0]):
          if cluster[i] == j:
            cluster_aux.append(x[i])

        # Now we store the "k_th" new centroid value, calculated as average of, x, y , z
        #print(np.asanyarray(cluster_aux).shape)
        new_centroids[j] = np.average(cluster_aux, axis = 0)

      #new_centroids = pd.DataFrame(X).groupby(by=cluster).mean().values

      # if centroids are same then leave
      if np.count_nonzero(centroids-new_centroids) == 0:
        diff = 0
      else:
        centroids = new_centroids
    return centroids, cluster

centroids, clusters = my_kmeans(point_cloud_array, 6)

print(centroids)
print(clusters)

draw_labels_on_model(point_cloud, clusters)

[[-1.59993286e+00  1.43914938e-03 -2.70437226e-03]
 [-3.99927033e+00 -4.00136124e+00 -9.57348792e-04]
 [ 3.58221156e+00 -1.41125937e-01  6.83996855e-01]
 [ 4.41326123e+00  1.26340945e-01 -6.87885334e-01]
 [ 4.00029464e+00 -3.99785507e+00  3.68433286e-04]
 [ 1.91258114e-02 -3.99959304e+00  1.10932522e-03]]
[0. 0. 0. ... 4. 4. 4.]
